<a href="https://colab.research.google.com/github/RavenWolfs/Sains-Data/blob/main/Task_Priority_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
File ini mendemonstrasikan proses pembuatan dan pelatihan model Machine Learning
sederhana untuk memprediksi skor prioritas sebuah tugas.
Model ini bisa menjadi otak di balik aplikasi Task Prioritizer.
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

def generate_sample_data(num_samples=1000):
    """
    Membuat dataset sampel untuk tugas harian.
    Fitur: Urgensi, Pentingnya, Perkiraan Waktu (jam), Kompleksitas.
    Target: Skor Prioritas.
    """
    np.random.seed(42)
    data = {
        'urgency': np.random.randint(1, 6, size=num_samples),      # Skala 1-5 (Sangat Rendah - Sangat Mendesak)
        'importance': np.random.randint(1, 6, size=num_samples),  # Skala 1-5 (Sangat Tidak Penting - Sangat Penting)
        'estimated_time': np.random.uniform(0.5, 8.0, size=num_samples).round(1), # Waktu dalam jam
        'complexity': np.random.randint(1, 6, size=num_samples)  # Skala 1-5 (Sangat Mudah - Sangat Rumit)
    }
    df = pd.DataFrame(data)

    # --- Logika Penentuan Skor Prioritas (Target Variable) ---
    # Ini adalah "aturan bisnis" yang akan kita ajarkan pada model.
    # Skor yang lebih tinggi berarti prioritas lebih tinggi.
    df['priority_score'] = (df['urgency'] * 3.5) + (df['importance'] * 2.5) - (df['estimated_time'] * 0.5) - (df['complexity'] * 0.2)

    # Menambahkan sedikit noise agar data lebih realistis
    noise = np.random.normal(0, 2, size=num_samples)
    df['priority_score'] += noise

    # Normalisasi skor agar berada dalam rentang yang wajar (misal: 0-100)
    min_score = df['priority_score'].min()
    max_score = df['priority_score'].max()
    df['priority_score'] = 100 * (df['priority_score'] - min_score) / (max_score - min_score)

    return df

def train_model(df):
    """
    Melatih model Regresi untuk memprediksi 'priority_score'.
    """
    # Memisahkan fitur (X) dan target (y)
    X = df[['urgency', 'importance', 'estimated_time', 'complexity']]
    y = df['priority_score']

    # Membagi data menjadi data latih dan data uji
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Data latih:", X_train.shape)
    print("Data uji:", X_test.shape)

    # Memilih dan melatih model (Random Forest Regressor adalah pilihan yang baik)
    model = RandomForestRegressor(n_estimators=100, random_state=42, oob_score=True)
    model.fit(X_train, y_train)

    # Mengevaluasi model
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"\nMean Squared Error (MSE) pada data uji: {mse:.2f}")
    print(f"Out-of-Bag (OOB) Score: {model.oob_score_:.4f}") # Mirip akurasi

    # Menampilkan pentingnya setiap fitur
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    print("\nPentingnya Fitur:")
    print(feature_importances)

    return model

def save_model(model, filename="task_priority_model.joblib"):
    """
    Menyimpan model yang sudah dilatih ke dalam sebuah file.
    """
    joblib.dump(model, filename)
    print(f"\nModel telah disimpan sebagai '{filename}'")

def load_and_predict(data_baru, filename="task_priority_model.joblib"):
    """
    Memuat model dan menggunakannya untuk prediksi pada data baru.
    """
    try:
        loaded_model = joblib.load(filename)
        skor_prediksi = loaded_model.predict(data_baru)
        return skor_prediksi
    except FileNotFoundError:
        print(f"Error: File model '{filename}' tidak ditemukan. Latih dan simpan model terlebih dahulu.")
        return None

# --- ALUR UTAMA ---
if __name__ == "__main__":
    # 1. Buat data sampel
    dataset = generate_sample_data()
    print("Contoh Data Sampel:")
    print(dataset.head())
    print("\n" + "="*50 + "\n")

    # 2. Latih model dari data
    trained_model = train_model(dataset)

    # 3. Simpan model
    save_model(trained_model)

    # 4. Contoh penggunaan model untuk prediksi tugas baru
    print("\n" + "="*50 + "\n")
    print("Memprediksi prioritas untuk tugas baru...")

    tugas_baru_1 = pd.DataFrame({
        'urgency': [5],       # Sangat Mendesak
        'importance': [5],    # Sangat Penting
        'estimated_time': [1],  # 1 jam
        'complexity': [2]     # Mudah
    })

    tugas_baru_2 = pd.DataFrame({
        'urgency': [2],       # Kurang Mendesak
        'importance': [3],    # Cukup Penting
        'estimated_time': [6],  # 6 jam
        'complexity': [5]     # Sangat Rumit
    })

    prediksi_1 = load_and_predict(tugas_baru_1)
    if prediksi_1 is not None:
        print(f"Prediksi Skor Prioritas Tugas 1 (seharusnya tinggi): {prediksi_1[0]:.2f}")

    prediksi_2 = load_and_predict(tugas_baru_2)
    if prediksi_2 is not None:
        print(f"Prediksi Skor Prioritas Tugas 2 (seharusnya rendah): {prediksi_2[0]:.2f}")

Contoh Data Sampel:
   urgency  importance  estimated_time  complexity  priority_score
0        4           4             5.1           5       56.630101
1        5           4             4.6           3       76.755758
2        3           4             2.2           1       62.240309
3        5           4             3.4           1       76.695691
4        5           5             5.0           1       91.074843


Data latih: (800, 4)
Data uji: (200, 4)

Mean Squared Error (MSE) pada data uji: 40.73
Out-of-Bag (OOB) Score: 0.8772

Pentingnya Fitur:
urgency           0.586231
importance        0.306712
estimated_time    0.085234
complexity        0.021823
dtype: float64

Model telah disimpan sebagai 'task_priority_model.joblib'


Memprediksi prioritas untuk tugas baru...
Prediksi Skor Prioritas Tugas 1 (seharusnya tinggi): 93.40
Prediksi Skor Prioritas Tugas 2 (seharusnya rendah): 32.24
